# 02 - Embeddings Comparison

This notebook compares three embedding approaches for RAG:

1. **OpenAI Embeddings** - High quality, API-based
2. **HuggingFace Embeddings (Legacy)** - sentence-transformers/all-MiniLM-L6-v2
3. **HuggingFace BGE** - BAAI/bge-small-en-v1.5 (upgraded model)

We'll create FAISS vector stores and compare performance and quality.

**Prerequisites:**
- 01_setup_and_basics.ipynb completed
- OpenAI API key configured

**Duration:** ~10 minutes

**Outputs:**
- Vector stores saved to `data/vector_stores/`
- Performance comparison metrics

## 1. Setup

Load necessary modules and prepare documents/chunks from previous notebook.

In [22]:
import sys
sys.path.append('../..')

from shared.config import OPENAI_VECTOR_STORE_PATH, HF_VECTOR_STORE_PATH
from shared.loaders import load_and_split
from shared.utils import print_section_header, save_vector_store
import time

print_section_header("Loading Documents and Chunks")

# Load and split documents
docs, chunks = load_and_split(verbose=True)

print(f"\n✅ Ready with {len(docs)} documents and {len(chunks)} chunks")


LOADING DOCUMENTS AND CHUNKS

Loading 4 documents from web...
  - https://python.langchain.com/docs/use_cases/question_answering/
  - https://python.langchain.com/docs/modules/data_connection/retrievers/
  - https://python.langchain.com/docs/modules/model_io/llms/
  - https://python.langchain.com/docs/use_cases/chatbots/
✓ Loaded 4 documents
✓ Added custom metadata to all documents
Splitting documents...
  - Chunk size: 1000
  - Chunk overlap: 200
✓ Created 120 chunks

  Sample chunk:
    - Length: 839 chars
    - Source: https://python.langchain.com/docs/use_cases/question_answering/
    - Preview: Build a RAG agent with LangChain - Docs by LangChainSkip to main contentWe've raised a $125M Series B to build the platform for agent engineering. Rea...

✅ Ready with 4 documents and 120 chunks


## 2. OpenAI Embeddings

### Features
- Model: `text-embedding-3-small`
- Dimensions: 1536
- Cost: ~$0.02 per 1M tokens
- Quality: Excellent

### Use When
- Production quality required
- Budget available
- Internet connection reliable

In [23]:
from langchain_openai import OpenAIEmbeddings

print_section_header("OpenAI Embeddings")

# Initialize OpenAI embeddings
print("Initializing OpenAI embeddings...")
openai_embeddings = OpenAIEmbeddings()
print("✓ OpenAI embeddings initialized")

# Test with a sample query
test_query = "What is retrieval-augmented generation?"
print(f"\nTesting with query: '{test_query}'")

start_time = time.time()
test_embedding = openai_embeddings.embed_query(test_query)
elapsed = time.time() - start_time

print("\nResults:")
print(f"  Dimension: {len(test_embedding)}")
print(f"  Time: {elapsed:.3f}s")
print(f"  First 5 values: {[f'{v:.4f}' for v in test_embedding[:5]]}")


OPENAI EMBEDDINGS

Initializing OpenAI embeddings...
✓ OpenAI embeddings initialized

Testing with query: 'What is retrieval-augmented generation?'

Results:
  Dimension: 1536
  Time: 0.241s
  First 5 values: ['-0.0403', '-0.0036', '0.0001', '0.0009', '-0.0103']


## 3. HuggingFace Embeddings

### Features
- Model: `sentence-transformers/all-MiniLM-L6-v2`
- Dimensions: 384
- Cost: Free (runs locally)
- Quality: Very good

### Use When
- Privacy is critical
- Offline operation needed
- Cost is a constraint
- Development/testing

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
import os

print_section_header("HuggingFace Embeddings")

print("Initializing HuggingFace embeddings...")
print("(First run downloads model ~90MB - may take 1-2 minutes)")
print(f"Cache location: {os.path.expanduser('~/.cache/huggingface/')}\n")

try:
    hf_embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    print("✓ HuggingFace embeddings initialized")
    
    # Test with same query
    print(f"\nTesting with query: '{test_query}'")
    
    start_time = time.time()
    test_embedding_hf = hf_embeddings.embed_query(test_query)
    elapsed_hf = time.time() - start_time
    
    print("\nResults:")
    print(f"  Dimension: {len(test_embedding_hf)}")
    print(f"  Time: {elapsed_hf:.3f}s")
    print(f"  First 5 values: {[f'{v:.4f}' for v in test_embedding_hf[:5]]}")
    
except Exception as e:
    print(f"✗ Error: {e}")
    print("\nTroubleshooting:")
    print("  1. Check internet connection (first run only)")
    print("  2. Verify disk space (~200MB needed)")
    print("  3. Try: pip install --upgrade sentence-transformers")
    raise


HUGGINGFACE EMBEDDINGS

Initializing HuggingFace embeddings...
(First run downloads model ~90MB - may take 1-2 minutes)
Cache location: /Users/gianlucamazza/.cache/huggingface/



## 3.5. HuggingFace BGE (Upgraded)

### Features
- Model: `BAAI/bge-small-en-v1.5` (BGE = BAAI General Embedding)
- Dimensions: 384 (same as all-MiniLM)
- Cost: Free (runs locally)
- Quality: **Superior** - competitive with OpenAI

### Improvements over all-MiniLM-L6-v2
- Better semantic understanding
- Optimized specifically for retrieval tasks
- Top performance on MTEB benchmark
- Same dimensions = drop-in replacement

### Use When
- Same as all-MiniLM, but with better quality
- **Recommended** for new projects

In [ ]:
print_section_header("HuggingFace BGE Embeddings")

print("Initializing BGE embeddings...")
print("(First run downloads model ~130MB - may take 1-2 minutes)")
print(f"Cache location: {os.path.expanduser('~/.cache/huggingface/')}\n")

try:
    bge_embeddings = HuggingFaceEmbeddings(
        model_name="BAAI/bge-small-en-v1.5"
    )
    print("✓ BGE embeddings initialized")
    
    # Test with same query
    print(f"\nTesting with query: '{test_query}'")
    
    start_time = time.time()
    test_embedding_bge = bge_embeddings.embed_query(test_query)
    elapsed_bge = time.time() - start_time
    
    print("\nResults:")
    print(f"  Dimension: {len(test_embedding_bge)}")
    print(f"  Time: {elapsed_bge:.3f}s")
    print(f"  First 5 values: {[f'{v:.4f}' for v in test_embedding_bge[:5]]}")
    
    print("\n💡 BGE is now the default HuggingFace model in config.py!")
    
except Exception as e:
    print(f"✗ Error: {e}")
    print("\nTroubleshooting:")
    print("  1. Check internet connection (first run only)")
    print("  2. Verify disk space (~300MB needed)")
    print("  3. Try: pip install --upgrade sentence-transformers")
    raise


HUGGINGFACE BGE EMBEDDINGS

Initializing BGE embeddings...
(First run downloads model ~130MB - may take 1-2 minutes)
Cache location: /Users/gianlucamazza/.cache/huggingface/



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✓ BGE embeddings initialized

Testing with query: 'What is retrieval-augmented generation?'

Results:
  Dimension: 384
  Time: 0.024s
  First 5 values: ['-0.0331', '0.0233', '-0.0349', '-0.0267', '0.0155']

💡 BGE is now the default HuggingFace model in config.py!


## 4. Side-by-Side Comparison

Now let's compare all three embedding models to see their differences in quality, speed, and characteristics.

In [ ]:
from shared.utils import print_comparison_table
import numpy as np

print_section_header("Embeddings Comparison")

data = [
    ["Feature", "OpenAI", "HF (Legacy)", "BGE (New)"],
    ["Model", "text-emb-3-small", "all-MiniLM-L6-v2", "bge-small-en-v1.5"],
    ["Dimension", len(test_embedding), len(test_embedding_hf), len(test_embedding_bge)],
    ["Time (s)", f"{elapsed:.3f}", f"{elapsed_hf:.3f}", f"{elapsed_bge:.3f}"],
    ["Mean", f"{np.mean(test_embedding):.4f}", f"{np.mean(test_embedding_hf):.4f}", f"{np.mean(test_embedding_bge):.4f}"],
    ["Std Dev", f"{np.std(test_embedding):.4f}", f"{np.std(test_embedding_hf):.4f}", f"{np.std(test_embedding_bge):.4f}"],
    ["Cost", "Paid", "Free", "Free"],
    ["Privacy", "Cloud", "Local", "Local"],
    ["Quality", "Excellent", "Very Good", "Excellent"]
]

print_comparison_table(data)

print("\n💡 Key Takeaways:")
print("   - OpenAI: Highest dimension (1536), cloud-based, paid")
print("   - HF Legacy: Good quality, local, free")
print("   - BGE: BEST VALUE - OpenAI-level quality, local, free!")
print("   - BGE is now the default in this tutorial")

## 5. Create Vector Stores

Now we'll create FAISS vector stores for OpenAI and BGE embeddings. These will be saved for reuse in all subsequent notebooks.

**Note:** We're using BGE (not all-MiniLM-L6-v2) as the default HuggingFace model going forward.

In [ ]:
from langchain_community.vectorstores import FAISS

print_section_header("Creating Vector Stores")

# Create OpenAI vector store
print("Creating FAISS vector store with OpenAI embeddings...")
start_time = time.time()
vectorstore_openai = FAISS.from_documents(chunks, openai_embeddings)
elapsed_openai = time.time() - start_time

print(f"✓ OpenAI vector store created in {elapsed_openai:.2f}s")
print(f"  - {len(chunks)} documents indexed")
print("  - Embedding dimension: 1536")

# Create BGE vector store (now the default HuggingFace model)
print("\nCreating FAISS vector store with BGE embeddings...")
start_time = time.time()
vectorstore_hf = FAISS.from_documents(chunks, bge_embeddings)
elapsed_bge_vs = time.time() - start_time

print(f"✓ BGE vector store created in {elapsed_bge_vs:.2f}s")
print(f"  - {len(chunks)} documents indexed")
print("  - Embedding dimension: 384")
print("  - Model: BAAI/bge-small-en-v1.5")

print("\n📊 Performance:")
print(f"   OpenAI: {elapsed_openai:.2f}s")
print(f"   BGE: {elapsed_bge_vs:.2f}s")
print(f"   BGE is {elapsed_openai/elapsed_bge_vs:.1f}x faster!")

## 6. Save Vector Stores

**IMPORTANT:** We save the vector stores to disk to avoid re-embedding in every notebook. This:
- Saves time (~3-4 seconds per notebook)
- Reduces API costs (OpenAI charges per embedding)
- Ensures consistency across notebooks

In [ ]:
print_section_header("Saving Vector Stores")

# Save OpenAI vector store
save_vector_store(vectorstore_openai, OPENAI_VECTOR_STORE_PATH, verbose=True)

# Save HuggingFace vector store
save_vector_store(vectorstore_hf, HF_VECTOR_STORE_PATH, verbose=True)

print("\n✅ Vector stores saved successfully!")
print("\n📂 Locations:")
print(f"   OpenAI: {OPENAI_VECTOR_STORE_PATH}")
print(f"   HuggingFace: {HF_VECTOR_STORE_PATH}")
print("\n💡 These will be loaded in subsequent notebooks to avoid re-embedding.")


SAVING VECTOR STORES

✓ Saved vector store to /Users/gianlucamazza/Workspace/notebooks/llm_rag/notebooks/fundamentals/../../data/vector_stores/openai_embeddings
✓ Saved vector store to /Users/gianlucamazza/Workspace/notebooks/llm_rag/notebooks/fundamentals/../../data/vector_stores/huggingface_embeddings

✅ Vector stores saved successfully!

📂 Locations:
   OpenAI: /Users/gianlucamazza/Workspace/notebooks/llm_rag/notebooks/fundamentals/../../data/vector_stores/openai_embeddings
   HuggingFace: /Users/gianlucamazza/Workspace/notebooks/llm_rag/notebooks/fundamentals/../../data/vector_stores/huggingface_embeddings

💡 These will be loaded in subsequent notebooks to avoid re-embedding.


## 7. Test Similarity Search

Quick test to verify the vector stores work correctly and compare retrieval quality.

In [ ]:
from shared.utils import print_results

print_section_header("Testing Similarity Search")

query = "How to build a RAG agent with LangChain?"
print(f"Query: '{query}'\n")

# Test OpenAI
print("=" * 80)
print("=== OpenAI Embeddings ===")
print("=" * 80)
results_openai = vectorstore_openai.similarity_search(query, k=3)
print_results(results_openai, max_docs=3, preview_length=150)

# Test BGE
print("\n" + "=" * 80)
print("=== BGE Embeddings (BAAI/bge-small-en-v1.5) ===")
print("=" * 80)
results_bge = vectorstore_hf.similarity_search(query, k=3)
print_results(results_bge, max_docs=3, preview_length=150)

print("\n" + "=" * 80)
print("\n✅ Both vector stores working correctly!")
print("\n💡 Notice: BGE retrieves high-quality results competitive with OpenAI!")

## Summary

In this notebook, we:

✅ Compared three embedding models (OpenAI, HF Legacy, BGE)  
✅ Created FAISS vector stores with BGE as default  
✅ Saved vector stores to disk for reuse  
✅ Tested similarity search quality  

### Key Takeaways

| Feature | OpenAI | HF Legacy | BGE (New) |
|---------|--------|-----------|-----------|
| Quality | Excellent | Very Good | **Excellent** |
| Dimension | 1536 | 384 | 384 |
| Cost | $0.02/1M | Free | **Free** |
| Privacy | Cloud | Local | **Local** |
| Speed | API latency | Fast | **Fast** |

### Recommendation

- **Best Value:** **BGE** (OpenAI quality, free, local) ⭐⭐⭐
- **Production (Cloud):** OpenAI (established, reliable)
- **Privacy-sensitive:** BGE or HF (data stays local)
- **Offline:** BGE or HF (no internet after download)
- **Legacy projects:** all-MiniLM-L6-v2 (still good)

### Upgrade Path

The tutorial now uses **BGE** as the default HuggingFace model:
- ✅ Better quality than all-MiniLM-L6-v2
- ✅ Same 384 dimensions (compatible)
- ✅ Drop-in replacement
- ✅ Configured in `shared/config.py`
- ✅ Vector stores now use BGE

### Next Steps

Continue to **[03_simple_rag.ipynb](03_simple_rag.ipynb)** to:
- Create retrievers from vector stores
- Build complete RAG chains
- Compare retrieval strategies (Similarity vs MMR)
- Evaluate RAG performance

---

**💾 Important:** Vector stores now use **BGE embeddings** (BAAI/bge-small-en-v1.5) and are ready for all subsequent notebooks!